In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
import pyrtools as pt
import plenoptic as po
import scipy.io as sio
import seaborn as sns
import h5py

torch.__version__
numpy = lambda x: x.detach().cpu().numpy().squeeze()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def relTol(a,b):
    return(abs(a-b)/(abs(a)+abs(b)))

def absTol(a,b):
    return(abs(a-b))

def reportTol(a,b):
    return (max(relTol(a,b)),max(absTol(a,b)))


In [2]:
f = h5py.File('/Users/kathrynbonnen/Documents/MATLAB/textureSynth/params.mat','r')
params=f['params']

f = h5py.File('/Users/kathrynbonnen/Documents/MATLAB/textureSynth/parents.mat','r')
rparents=np.transpose(f['rparents'][()],(1,0)).astype(np.float32)
cousins = np.transpose(f['cousins'][()],(1,0)).astype(np.float32)
cousinSz = f['cousinSz'][()]



In [3]:
print(params.keys())


<KeysViewHDF5 ['autoCorrMag', 'autoCorrReal', 'cousinMagCorr', 'cousinRealCorr', 'magMeans', 'parentMagCorr', 'parentRealCorr', 'pixelLPStats', 'pixelStats', 'varianceHPR']>


In [4]:
statg0_m =  params['pixelStats'][()];
magMeans0_m =  params['magMeans'][()];
ace_m = np.transpose(params['autoCorrMag'][()],(3,2,1,0)).astype(np.float32);
lpstats = np.transpose(params['pixelLPStats'][()],(1,0)).astype(np.float32);
skew0p_m = lpstats[:,0]
kurt0p_m = lpstats[:,1]
vHPR0_m = params['varianceHPR'][()];
acr_m = np.transpose(params['autoCorrReal'][()],(2,1,0)).astype(np.float32);
C0_m = np.transpose(params['cousinMagCorr'][()],(2,1,0)).astype(np.float32);
Cx0_m = np.transpose(params['parentMagCorr'][()],(2,1,0)).astype(np.float32);
Cr0_m = np.transpose(params['cousinRealCorr'][()],(2,1,0)).astype(np.float32);
Crx0_m = np.transpose(params['parentRealCorr'][()],(2,1,0)).astype(np.float32);

In [5]:
# image = plt.imread('../data/flowers.jpg').astype(float).mean(axis=2)
image = plt.imread('../data/metal.pgm').astype(float)
image = image[0:64,0:64]
im0 = torch.tensor(image,requires_grad=True,dtype = torch.float32).squeeze().unsqueeze(0).unsqueeze(0)
model = po.simulate.Portilla_Simoncelli(im0.shape[-2:],n_scales=4,n_orientations=4,Na=3)
y = model.forward(im0)

statg0:  torch.Size([6, 1]) 6
magMeans0:  torch.Size([18, 1]) 18
ace:  torch.Size([3, 3, 4, 4]) 144
skew0p:  torch.Size([5, 1]) 5
kurt0p:  torch.Size([5, 1]) 5
acr:  torch.Size([3, 3, 5]) 45
C0:  torch.Size([4, 4, 5]) 80
Cx0:  torch.Size([4, 4, 4]) 64
Cr0:  torch.Size([8, 8, 5]) 320
Crx0:  torch.Size([8, 8, 4]) 256
vHPR0:  tensor(384.4385, grad_fn=<MeanBackward0>) 1


In [6]:
a = 0
b = 1023
print((a,b))
n1=np.matmul(cousins[a:b,0].T,rparents[a:b,0])/cousinSz
n2=np.matmul(cousins[a:b,0].T,model.rparents[a:b,0])/model.cousinSz

print(reportTol(n1.flatten(),n2.flatten()))
print(n1.T)
print(n2.T)

(0, 1023)
(1.0, 0.02337646484375)
[[-0.00146484]]
0.02191162109375


In [7]:
print(reportTol(statg0_m.flatten(),model.statg0.flatten()))

print(reportTol(magMeans0_m.flatten(),model.magMeans0.flatten()))

print(reportTol(ace_m.flatten(),model.ace.flatten()))

print(reportTol(skew0p_m.flatten(),model.skew0p.flatten()))

print(reportTol(kurt0p_m.flatten(),model.kurt0p.flatten()))

print(reportTol(acr_m.flatten(),model.acr.flatten()))

print(reportTol(C0_m.flatten(),model.C0.flatten()))

print(reportTol(Cx0_m.flatten(),model.Cx0.flatten()))

print(reportTol(Cr0_m.flatten(),model.Cr0.flatten()))

print(reportTol(Crx0_m.flatten(),model.Crx0.flatten()))

print(reportTol(vHPR0_m.flatten(),model.vHPR0.flatten()))


(4.174799255011961e-08, 2.4086156827252125e-05)
(9.847755537416507e-08, 0.00021234479072518297)
(2.1983153e-07, 0.0625)
(9.647189e-07, 6.556511e-07)
(1.4783791e-07, 9.536743e-07)
(3.4270357e-07, 0.75)
(2.7216592e-07, 0.03125)
(3.4875286e-06, 0.002319336)
(2.627582e-06, 0.375)
(1.0, 0.078125)
(9.639355384938505e-08, 7.411480089558609e-05)


/Users/kathrynbonnen/anaconda3/envs/plenoptic_docs/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


In [8]:
s = po.simulate.Steerable_Pyramid_Freq(image.shape, 4,
            3, is_complex=True, store_unoriented_bands=True,
                                          return_list=True)

In [32]:
pyr = s.forward(im0)

In [33]:
pyr[1].shape

torch.Size([1, 1, 256, 256, 2])

In [34]:
tmp = sio.loadmat('../data/plenoptic-test-files/PortillaSimoncelliMatlab0.mat')

In [35]:
matfile = sio.loadmat('../data/plenoptic-test-files/PortillaSimoncelliMatlab'+str(0))
im0 = torch.Tensor(matfile['im0']).unsqueeze(0).unsqueeze(0)
paramsMat = np.array(matfile['paramsVector']).flatten()
model = po.simulate.Portilla_Simoncelli(im0.shape[-2:],n_scales=4,n_orientations=4,Na=9)
paramsPython = model.forward(im0)
paramsPython = po.to_numpy(paramsPython)

# print(paramsPython.shape)
# print(paramsMat.shape)
# print(abs(paramsMat-paramsPython))

print(paramsMat[1384])
print(paramsPython[1384])
print(np.where(abs(paramsMat-paramsPython)>30))


torch.Size([32, 32, 2])
tensor([[   22.4766,   339.4022,   457.5108,  ...,  -339.4024,  -457.5112,
          -307.6158],
        [  339.4023,  5125.1748,  6908.6890,  ..., -5125.1748, -6908.6895,
         -4645.1875],
        [  457.5107,  6908.6895,  9312.8516,  ..., -6908.6895, -9312.8516,
         -6261.6714],
        ...,
        [ -339.4023, -5125.1748, -6908.6895,  ...,  5125.1748,  6908.6890,
          4645.1875],
        [ -457.5108, -6908.6895, -9312.8516,  ...,  6908.6895,  9312.8516,
          6261.6714],
        [ -307.6156, -4645.1875, -6261.6714,  ...,  4645.1875,  6261.6709,
          4210.1533]])
statg0:  torch.Size([6, 1]) 6
magMeans0:  torch.Size([18, 1]) 18
ace:  torch.Size([9, 9, 4, 4]) 1296
skew0p:  torch.Size([5, 1]) 5
kurt0p:  torch.Size([5, 1]) 5
acr:  torch.Size([9, 9, 5]) 405
C0:  torch.Size([4, 4, 5]) 80
Cx0:  torch.Size([4, 4, 4]) 64
Cr0:  torch.Size([8, 8, 5]) 320
Crx0:  torch.Size([8, 8, 4]) 256
vHPR0:  tensor(74.0163) 1
1.4901161193847656e-08
32.0
(array(